#  Fine-tuning of Roberta - SPAM language model
link to model: https://huggingface.co/mshenoda/roberta-spam



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


Installation of necessary libraries

In [2]:
!pip install -U accelerate -q
!pip install -U transformers -q
!pip install -U datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 29.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.3 MB/s eta 0:00:00


Library import and version check

In [3]:
import transformers
transformers.__version__

'4.36.2'

## Loading previously cleared SMS message data


In [4]:
import pandas as pd

data = pd.read_csv('train.csv')
data.sample(3)

,sms,label
412,Cos i want it to be your thing\n,0
2609,Hello madam how are you ?\n,0
1967,What's a feathery bowa? Is that something guys...,0


In [5]:
data.to_feather

<bound method DataFrame.to_feather of                                                     sms  label
0     Go until jurong point, crazy.. Available only ...      0
1                       Ok lar... Joking wif u oni...\n      0
2     Free entry in 2 a wkly comp to win FA Cup fina...      1
3     U dun say so early hor... U c already then say...      0
4     Nah I don't think he goes to usf, he lives aro...      0
...                                                 ...    ...
5569  This is the 2nd time we have tried 2 contact u...      1
5570             Will ü b going to esplanade fr home?\n      0
5571  Pity, * was in mood for that. So...any other s...      0
5572  The guy did some bitching but I acted like i'd...      0
5573                       Rofl. Its true to its name\n      0

[5574 rows x 2 columns]>

Import the Dataset library and convert the table to a Dataset object. Then split the data.

In [6]:
from datasets import Dataset

dataset_ = Dataset.from_pandas(data)
dataset = dataset_.train_test_split(0.1)

In [7]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['sms', 'label'],
        num_rows: 5016
    })
    test: Dataset({
        features: ['sms', 'label'],
        num_rows: 558
    })
})


## Fine-tuning

First, the name of the model used was entered, i.e.: mshenoda/robert-spam, and the batch_size was specified.

In [8]:
model_checkpoint = 'mshenoda/roberta-spam'
batch_size = 16

Next, import the AutoTokenizer class from the transformers library and use the from_pretrained method to load the tokenizer into the tokenizer.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/999k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/957 [00:00<?, ?B/s]

The method downloaded from the Internet a fragment of the model responsible for tokenization and created a method to convert text data into a list of indexes understood by the model.

In [10]:
tokenizer("This is a sample")

{'input_ids': [0, 713, 16, 10, 7728, 2], 'attention_mask': [1, 1, 1, 1, 1, 1]}

Creating the process method and using the map method

In [11]:
def process(x):
  return tokenizer(x['sms'])

train_ds = dataset['train'].map(process)
eval_ds = dataset['test'].map(process)

Map:   0%|          | 0/5016 [00:00<?, ? examples/s]

Map:   0%|          | 0/558 [00:00<?, ? examples/s]

In [12]:
train_ds

Dataset({
    features: ['sms', 'label', 'input_ids', 'attention_mask'],
    num_rows: 5016
})

### Training



In [13]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

num_labels = 2
model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=num_labels)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/735 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Setting the training parameters.
Created an object of class TrainingArguments to the variable args and set the arguments one by one:

* the first argument is some custom model name - it's worth referring to `model_checkpoint`, but in practice it's any string.
* `evaluation_strategy` set to `epoch`, i.e. evaluation after each completed epoch.
* `save_strategy` set to `'epoch` similarly to the previous one.
* `learning_rate` set to `2e-5` - this can of course ultimately be tuned.
* `per_device_train_batch_size` set to `batch_size` defined above.
* `per_device_eval_batch_size` also set to `batch_size`.
* `num_train_epochs` set to 5 for now.
* `weight_decay` set as `0.01` - this can also be tuned.
* `load_best_model_at_end` set to `True` - this is useful for us.
* `metric_for_best_model` set to `accuracy`.

In [14]:
args = TrainingArguments(
    f'{model_checkpoint}_sentiment_analysis',
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    learning_rate = 2e-5,
    per_device_train_batch_size = batch_size,
    per_device_eval_batch_size = batch_size,
    num_train_epochs = 5,
    weight_decay = 0.01,
    load_best_model_at_end = True,
    metric_for_best_model = 'accuracy'
)

Creating a method to count quality metrics.

In [15]:
from datasets import load_metric
import numpy as np

metric = load_metric('glue', 'sst2')

def compute_metrics(eval_preds):
  logits, labels = eval_preds
  predictions = np.argmax(logits, axis=-1)
  return metric.compute(predictions=predictions, references=labels)

<ipython-input-15-35f6af1b57cb>:4: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric('glue', 'sst2')
/usr/local/lib/python3.10/dist-packages/datasets/load.py:752: FutureWarning: The repository for glue contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.16.1/metrics/glue/glue.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


Created a Trainer class object to the trainer variable and set the arguments accordingly:

* first argument is the model loaded above
* the second argument is the args containing the trainer options created above
* set train_dataset as train_ds
* set eval_dataset to test_ds
* tokenizer set to tokenizer
* compute_metrics set to the `compute _metrics method

In [16]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_ds,
    eval_dataset=eval_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics
)

Starting iteration of evaluation using trainer.evaluate method.

In [17]:
trainer.evaluate([train_ds[0]])

You're using a RobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 1.7881377516459906e-06,
 'eval_accuracy': 1.0,
 'eval_runtime': 1.0141,
 'eval_samples_per_second': 0.986,
 'eval_steps_per_second': 0.986}

Training launch.

In [18]:
trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.009234,0.998208
2,0.015400,0.000013,1.000000
3,0.015400,0.000004,1.000000
4,0.005400,0.000003,1.000000
5,0.000000,0.000003,1.000000


TrainOutput(global_step=1570, training_loss=0.006646611946608995, metrics={'train_runtime': 383.4562, 'train_samples_per_second': 65.405, 'train_steps_per_second': 4.094, 'total_flos': 824758603018080.0, 'train_loss': 0.006646611946608995, 'epoch': 5.0})

## Testing the model

Reading the model.

In [19]:
trainer.model

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

Verification of model performance with preparation of input data

In [20]:
import torch

text = "Congratulations! You've won a free iPhone. Claim it now!"
inputs = tokenizer(text, return_tensors='pt')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

input_ids = inputs['input_ids'].to(device)
attention_mask = inputs['attention_mask'].to(device)

with torch.no_grad():
  outputs = model(input_ids=input_ids, attention_mask=attention_mask)
  logits = outputs.logits
  predictions = torch.argmax(logits, dim=-1)

if predictions.item() == 0:
  print("NOT SPAM")
else:
  print("SPAM")

SPAM


The model correctly classified the text entered about receiving a free iphone as spam.